Có những quá trình chuẩn trong machine learning có thể tự động hóa. Mà trong thư viện scikit learn, Pipelines giúp cho việc dễ dàng định ra và tự động hóa các quá trình này.

### I. Automating Machine Learning Workflows

### II. Data Preparation and Modeling Pipeline

Một lỗi dễ mắc phải khi ứng dụng machine learning là việc không đồng nhất tính chất giữa tập train và tập test. Nhất là nằm ở giai đoạn chuẩn bị dữ liệu. Ví dụ: việc toàn bộ tập train thì dùng một phương thức chuẩn hóa nào đó mà tập test thì không, dẫn đến việc học trên tập train nhưng không dự đoán đúng trên tập test bởi sự khác biệt giữa chuẩn tỷ lệ trong tập test.<br>
Pipelines giúp ngăn dữ liệu bị không đồng nhất bằng việc bảo đảm trong quá trình chuẩn bị dữ liệu như chuẩn hóa thì xảy ra đồng thời ở mọi fold khi cross validation.<br>
Pipeline ở ví dụ dưới làm 2 việc đó là chuẩn hóa dữ liệu và học mô hình LDA

In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = pd.read_csv('pima-indians-diabetes.csv', names=names)
array = dataframe.values
X = array[:, :8] # input
Y = array[:, 8] # output
dataframe.head()

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [16]:
# create pipeline
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('LDA', LinearDiscriminantAnalysis()))
model = Pipeline(estimators)
# evaluate pipeline
num_folds = 10
seed = 7
kfold = KFold(n_splits=num_folds, random_state=seed)
scoring = 'accuracy'
results = cross_val_score(model, X, Y, cv=kfold, scoring=scoring)
print(results.mean())

0.773462064251538


In [17]:
estimators

[('standardize', StandardScaler(copy=True, with_mean=True, with_std=True)),
 ('LDA',
  LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                             solver='svd', store_covariance=False, tol=0.0001))]

### III. Feature Extraction and Modeling Pipeline

FeatureUnion trong Pipeline. Ví dụ: Feature Extraction với PCA (n_components =3) và SelectKbest (k=6) là việc cho chạy song song 2 kĩ thuật feature selection sau đó nối các features lại với nhau (X mới có features 3+6=9)

In [24]:
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import LogisticRegression

# create feature union
features = []
features.append(('PCA', PCA(n_components=3)))
features.append(('Select K best', SelectKBest(k=6)))
feature_union = FeatureUnion(features)
# create pipeline
estimators = []
estimators.append(('feature union', feature_union))
estimators.append(('LR', LogisticRegression(solver='liblinear')))
model = Pipeline(estimators)
# evaluate pipeline
num_folds = 10
seed = 7
kfold = KFold(n_splits=num_folds, random_state=seed)
scoring = 'accuracy'
results = cross_val_score(model, X , Y, cv=kfold, scoring=scoring)
print(results.mean())

0.7760423786739576


**Note:** Việc dùng feature union ở ví dụ trên không phải để giảm chiều mà là để transform features - ở trường hợp này là tạo nhiều feature hơn. Bên dưới là các phần test thử:
- Không dùng kĩ thuật feature selection nào
- Dùng PCA decomposition với n_components = 5
- Dùng SelectKbest với k =5

In [28]:
# test thử với không sử dụng feature selection nào
# create pipeline
estimators = []
estimators.append(('LR', LogisticRegression(solver='liblinear')))
model = Pipeline(estimators)
kfold = KFold(n_splits=num_folds, random_state=seed)
scoring = 'accuracy'
results = cross_val_score(model, X , Y, cv=kfold, scoring=scoring)
print(results.mean())

0.7695146958304853


In [26]:
# test thử với PCA n_components = 5
# create pipeline
estimators = []
estimators.append(('PCA', PCA(n_components=5)))
estimators.append(('LR', LogisticRegression(solver='liblinear')))
model = Pipeline(estimators)
kfold = KFold(n_splits=num_folds, random_state=seed)
scoring = 'accuracy'
results = cross_val_score(model, X , Y, cv=kfold, scoring=scoring)
print(results.mean())

0.7513841421736159


In [27]:
# test thử với SelectKbest k = 5
# create pipeline
estimators = []
estimators.append(('Select K Best', SelectKBest(k=5)))
estimators.append(('LR', LogisticRegression(solver='liblinear')))
model = Pipeline(estimators)
kfold = KFold(n_splits=num_folds, random_state=seed)
scoring = 'accuracy'
results = cross_val_score(model, X , Y, cv=kfold, scoring=scoring)
print(results.mean())

0.7695488721804511


**Kết luận** ở trường hợp này dùng việc transform features - tạo nhiều features hơn thì đạt kết quả tốt hơn (**0.776**).